In [ ]:
import os
from os.path import basename, dirname, join, exists
import sys
import glob 

import numpy as np
import numpy.ma as ma


import matplotlib as mpl
import matplotlib.pyplot as plt



In [ ]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)
    
# dir of all gwfm data
gwfm_dir = join(dirname(doc_dir),'Box/research_cosumnes/GWFlowModel')


In [ ]:
# set box directory for output figures and data
box_dir = gwfm_dir+'/Levee_setback/levee_setback_distance_analysis/'

# tprogs_id = '' # original tprogs with conditioning data in output tsim
tprogs_id = '_no_conditioning'

data_dir = box_dir+ tprogs_id+'/data_output/'
fig_dir = box_dir+tprogs_id+'/figures/'

chan_dir = box_dir+'channel_data/'
gis_dir = chan_dir+'GIS/'

# Local model Connec3D example
Based on review it seems that Alisha may have done the input wrong by having an array formatted as z y x before converting it to 1D because the output of the CCO shows random CCO values when they should be grouped correctly onto the components.  
It also seems that just based on the shape of the coarse facies that Alisha converted the conditioning data (- values) into zeros instead of 1s as it should have been.

In [ ]:
c3d_dir = join(box_dir, 'connec3d_local_results')
os.listdir(c3d_dir)

In [ ]:
# x,y, z
nrow, ncol, nlay = 105, 60, 200

In [ ]:
# z, y,x is the shape that produces appropriate volumes
dat_in = np.loadtxt(join(c3d_dir, 'R11newww.dat'))


In [ ]:
arr_shape = (nlay,ncol,nrow) # only format to give expected units in all three directions
# arr_shape = (nrow, ncol, nlay) # totally wrong, creates very thin lines and weird staggers
# arr_shape = (nlay, nrow, ncol) # creates weird straight lines as well, looks normal in y-z slices

# z, y, x ordering
dat = np.reshape(dat_in, arr_shape)

# plt.imshow(dat[:,30,:])
# plt.imshow(dat[:,:,50])
plt.imshow(dat[50,:,:])
# almost looks like the location of wells was clipped



In [ ]:
arr_shape = (nlay,ncol,nrow) # gives normal body shape, but weird CCO number patterns
# arr_shape = (nrow, ncol, nlay) # bodies are scattered irregularly but CCO numbers are distinct
arr_shape = (nlay, nrow, ncol) # bodies are normal in XZ, weird elsewise and CCO numbers are off

cco_18_in = np.loadtxt(join(c3d_dir, 'R11_18.CCO'))
cco_6_in = np.loadtxt(join(c3d_dir, 'R11_6.CCO'))
# reshape to array format
cco_18 = np.reshape(cco_18_in, arr_shape)
cco_6 = np.reshape(cco_6_in, arr_shape)

In [ ]:
# in all directions it is unclear how the value of CCO is connected as it appears to be in straight lines
temp = np.ma.masked_where(cco_6==0,cco_6)
plt.imshow(temp[:,30,:])
# plt.imshow(temp[50])
# plt.imshow(temp[:,:,50])

In [ ]:
def cco_review(cco_in, name):
    print('For ',name,' pt %.2f %% are connected' %(100*np.sum(cco_in!=0)/ cco_in.shape[0]))
    n = np.unique(cco_in).shape[0]

    cnt = np.histogram(cco_in, bins = np.arange(1, n+1)-0.5)
    np.argmax(cnt[0])
    print('Largest ', name, ' pt body is %.2f %% of volume' %(100*np.max(cnt[0])/cco_in.shape[0]))

cco_review(cco_18_in, '18')
cco_review(cco_6_in, '6')

In [ ]:
k = 50
fig,ax =plt.subplots(1,2, figsize=(8,6))
im = ax[0].imshow(cco_18[:,:,k]>1)
# plt.colorbar(im)
ax[0].set_title('18 pt')
# plt.show()

im = ax[1].imshow(cco_6[:,:,k]>1)
# plt.colorbar(im)
ax[1].set_title('6 pt')
